<a href="https://colab.research.google.com/github/Pandatoey/LAB229351/blob/main/424_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cvxpy[SCIP]

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import numpy as np
import cvxpy as cp

db = load_diabetes()

x = db.data
y = db.target
names = db.feature_names

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [3]:
alphas = np.linspace(0.001,1,500)

for a in alphas:
    lasso = Lasso(alpha=a, max_iter=10000)
    lasso.fit(x_train, y_train)
    if np.sum(lasso.coef_ != 0) == 4:
        break

print("alpha =", a)
print("coef =", lasso.coef_)
print("Selected =", np.array(names)[lasso.coef_ != 0])

y_pred = lasso.predict(x_test)
print("Test MSE =", mean_squared_error(y_test, y_pred))


alpha = 1.0
coef = [  0.48287569 -10.23655929  26.90055639  17.34015649  -7.59247165
  -0.         -10.59008537   4.28698801  19.08227825   1.87752512]
Selected = ['age' 'sex' 'bmi' 'bp' 's1' 's3' 's4' 's5' 's6']
Test MSE = 2784.3343859801266


In [4]:
k = 4
M = 100
d = x_train.shape[1]

beta = cp.Variable(d)
beta0 = cp.Variable()
z = cp.Variable(d, boolean=True)

residual = y_train - (beta0 + x_train @ beta)
objective = cp.Minimize(cp.sum_squares(residual))

constraints = [
    cp.sum(z) <= k,
    beta <= M * z,
    beta >= -M * z
]

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

962097.1797796472

In [5]:
#MIP
print("beta =", beta.value)
print("beta0 =", beta0.value)

selected = np.abs(beta.value) > 1e-4
print("Selected =", np.array(names)[selected])

y_pred_mip = beta0.value + x_test @ beta.value
print("Test MSE =", mean_squared_error(y_test, y_pred_mip))


beta = [  0.           0.          28.81489897  16.23962172   0.
   0.         -12.17073344   0.          17.45348039   0.        ]
beta0 = 153.90291387765203
Selected = ['bmi' 'bp' 's3' 's5']
Test MSE = 2897.3615934751906
